In [ ]:
import flapjack
from flapjack import Flapjack
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

## Connecting to Flapjack

In [ ]:
fj = Flapjack(url_base='localhost:8000')
fj.log_in(username='tim', password='chicken')

In [ ]:
fj.get('sample')

In [ ]:
fj.patch('sample', 3, supplements=[1])

## Query Flapjack by filtering and searching

In [ ]:
study = fj.get('study', search='Phase')
assay = fj.get('assay', study=study.id[0])
sample = fj.get('sample', assay=assay.id[0])
chems = fj.get('chemical', sample=sample.id[0])
media = fj.get('media', search='glucosa')

od = fj.get('signal', name='OD')
cfp = fj.get('signal', search='CFP')

dna = fj.get('dna')

measurement = fj.get('measurement', sample=sample.id[10])

In [ ]:
study

In [ ]:
assay

In [ ]:
sample

In [ ]:
measurement

In [ ]:
fig = px.scatter(measurement, x='time', y='value', color='signal')
fig.show()
print(len(sample.id))
measurement['sample'].unique()

In [ ]:
chemical = fj.get('chemical', pubchemid=12345)
chemical

## Plot the data

In [ ]:
fig = fj.plot(assay=assay.id,
                    media=media.id,
                    normalize='None', 
                    subplots='Signal', 
                    markers='Vector', 
                    plot='Mean')

In [ ]:
fig.show()

## Analyse the data and plot the result

In [ ]:
fig = fj.plot(assay=assay.id,
                    media=media.id, 
                    type='Mean Expression',
                    biomass_signal=od.id,
                    ref_signal=cfp.id,
                    #bg_correction=2,
                    #min_biomass=0.05,
                    #remove_data=False,
                    normalize='None', 
                    subplots='Signal', 
                    markers='Vector', 
                    plot='Mean')

In [ ]:
fig.show()

## Analyze the data and get a dataframe of the result

In [ ]:
df = fj.analysis(assay=[assay.id[0]],
                    media=media.id, 
                    type='Alpha',
                    biomass_signal=od.id,
                    ref_signal=cfp.id,
                    #bg_correction=2,
                    #min_biomass=0.05,
                    #remove_data=False
                      )


### Custom plot the results

In [ ]:
px.bar(df, x='Vector', y='Alpha', facet_col='Signal')

### Statistical analysis of the results (TO DO)

## Compute an induction curve and fit a model

In [ ]:
dna = fj.get('dna', name='CcaT')
vector = fj.get('vector', dnas=dna.id)
chemical = fj.get('chemical', name='IPTG')
biomass_signal = fj.get('signal', name='OD')

df = fj.analysis(vector=vector.id,
                 analyte=chemical.id,
                 type='Induction Curve',
                 function='Mean Expression',
                 biomass_signal=biomass_signal.id
                )

### Fit a model to the data
The fit_curve function is a convenient wrapper for SciPy's curve_fit function. Here you may pass optional keyword arguments (such as p0, bounds) to curve_fit (see https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html). The function returns the fitted parameters and the estimated standard deviation.

Flapjack provides several common functions such as the Hill function used here, but you may specify any function just as with curve_fit. The parameters x and y are the columns that are used for the x and y values to fit the function.

In [ ]:
data = df[df.Signal=='Turquoise'][df.Vector=='CcaT+TMA5']
params,std = flapjack.fit_curve(flapjack.hill, data, x='Concentration', y='Expression')

In [ ]:
print(params)
print(std)

### Plot the result of the fit

In [ ]:
data.plot(x='Concentration', y='Expression', label='CcaT+TMA5', style='+', logx=True)
x = np.logspace(-6,-3,100)
y = flapjack.hill(x, *params)
plt.plot(x, y, 'k--')

## Connecting to SynbioHub

In [ ]:
from sbol import *

dna = fj.get('dna', name='pAAA')
sboluri = dna.sboluri

synbiohub = PartShop('https://synbiohub.org')
doc = Document()
synbiohub.pull(sboluri, doc)

## Creating new studies and associated assays

### Create a new study

In [ ]:
study = fj.create('study', name='Another test study', description='Testing the API again')
study

### Modify the study

In [ ]:
study = fj.patch('study', study.id[0], name='Changed name')

In [ ]:
study

### Add an assay to the study

In [ ]:
assay = fj.create('assay',
                  name='Test',
                  description='A test',
                  machine='Test',
                  temperature=12,
                  study=study.id[0]
                 )


In [ ]:
fj.get('assay')

In [ ]:
fj.delete('assay', assay.id[0])

In [ ]:
sample = fj.create('sample', )

### Get a DNA and associate it with a vector

In [ ]:
dnas = fj.get('dna', name='TestDNA')
vector = fj.create('vector', name='TestVector', dnas=dnas.id)

In [ ]:
vector

### Delete the objects we just created

In [ ]:
fj.delete('assay', assay.id[0])

This also deletes all the assays, samples and measurements in the study.

In [ ]:
fj.get('assay')

### Create a signal

In [ ]:
signal = fj.create('signal', name='TestFP', description='A test', color='green')
fj.get('signal')

In [ ]:
fj.patch('signal', signal.id[0], name='changed name')

### Delete the signal

In [ ]:
fj.delete('signal', signal.id[0])
fj.get('signal')

## Directly accessing raw measurements

In [ ]:
study = fj.get('study', search='Inducible')
vector = fj.get('vector', name='TheDNA2')
assay = fj.get('assay')
assay

In [ ]:
df = fj.measurements(assay=[4])

In [ ]:
fj.get('media')

In [ ]:
media = fj.get('media', name='Simulated media')
strain = fj.get('strain', name='Simulated strain')
vector = fj.get('vector', name='pSIM1')
study = fj.get('study', name='Inducible promoter simulation')
assay = fj.get('assay', name='Simulated inducer 1')
sample = fj.create('sample',
                        row=1, col=1,
                        media=media.id[0],
                        strain=strain.id[0],
                        vector=vector.id[0],
                        assay=assay.id[0],
                        supplements=[1]
                        )

In [ ]:
fj.delete('media', media.id[0])

In [ ]:
fj.get('study')